In [64]:
import cv2
import random
import shutil

In [3]:
cap = cv2.VideoCapture(0)
frame_count=0
while True:
    ret, frame = cap.read()

    if ret:
        cv2.imshow('Camera', frame)

    if cv2.waitKey(1)&0xFF == ord('q'):
        break

    file_name = f"frame_{frame_count}.jpg"
    cv2.imwrite(file_name, frame)

    frame_count += 1
cap.release()
cv2.destroyAllWindows()

In [44]:
# Create a directory to save the captured images
if not os.path.exists('captured_images'):
    os.makedirs('captured_images')
if not os.path.exists('captured_labels'):
    os.makedirs('captured_labels')
if not os.path.exists('labels'):
    os.makedirs('labels')

In [45]:
import cv2
import mediapipe as mp
import os
    
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

bboxes = []
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
  frame_count = 0  # Counter for captured frames
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image_rgb)

    # Draw the face detection annotations on the image.
    image.flags.writeable = True
    if results.detections:
      for idx, detection in enumerate(results.detections):
        bboxC = detection.location_data.relative_bounding_box
        ih, iw, _ = image.shape
        x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), 
                     int(bboxC.width * iw), int(bboxC.height * ih)
        
        # Assign different class_id for each bounding box
        class_id = idx  # Class_id starts from 0 for the first box
        
        x_center = (x + w / 2) / iw
        y_center = (y + h / 2) / ih
        bbox_width = w / iw
        bbox_height = h / ih
        yolo_format = f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}"
        
        # Draw bounding box and label on the image
        # label = f"Class {class_id}"  # You can customize this label as needed
        # cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        text_filename = f'captured_labels/frame_{frame_count}.txt'
        with open(text_filename, 'a') as f:
            f.write(yolo_format + '\n')
            
        image_filename = f'captured_images/frame_{frame_count}.jpg'
        cv2.imwrite(image_filename, flipped_image)
        print(f"Saved {image_filename}")
          
        print(f"YOLO Format BBox for frame {frame_count}: {yolo_format}")
    
    # Flip the image horizontally for a selfie-view display.
    flipped_image = cv2.flip(image, 1)
    cv2.imshow('MediaPipe Face Detection', flipped_image)
    
    # Save the captured frame as an image
    # if frame_count % 10 == 0:  # Save every 30 frames (adjust as needed)
        # image_filename = f'captured_images/frame_{frame_count}.jpg'
        # cv2.imwrite(image_filename, flipped_image)
        # print(f"Saved {image_filename}")
    
    if cv2.waitKey(5) & 0xFF == 27:
      break
    
    frame_count += 1

cap.release()
cv2.destroyAllWindows()


Saved captured_images/frame_0.jpg
YOLO Format BBox for frame 0: 0 0.710938 0.845833 0.306250 0.408333
Saved captured_images/frame_0.jpg
YOLO Format BBox for frame 0: 1 0.136719 0.886458 0.335938 0.447917
Saved captured_images/frame_1.jpg
YOLO Format BBox for frame 1: 0 0.707812 0.843750 0.303125 0.404167
Saved captured_images/frame_1.jpg
YOLO Format BBox for frame 1: 1 0.140625 0.883333 0.334375 0.445833
Saved captured_images/frame_2.jpg
YOLO Format BBox for frame 2: 0 0.707812 0.843750 0.309375 0.412500
Saved captured_images/frame_2.jpg
YOLO Format BBox for frame 2: 1 0.138281 0.888542 0.339062 0.452083
Saved captured_images/frame_3.jpg
YOLO Format BBox for frame 3: 0 0.706250 0.839583 0.300000 0.400000
Saved captured_images/frame_3.jpg
YOLO Format BBox for frame 3: 1 0.135156 0.890625 0.342187 0.456250
Saved captured_images/frame_4.jpg
YOLO Format BBox for frame 4: 0 0.711719 0.842708 0.301563 0.402083
Saved captured_images/frame_4.jpg
YOLO Format BBox for frame 4: 1 0.135156 0.89270

In [46]:
images = os.listdir('./captured_images')
labels = os.listdir('./captured_labels')

In [47]:
len(images)

241

In [48]:
len(labels)

241

In [51]:
random.shuffle(images)

train_ratio = 0.8
total_images = len(images)
train_count = int(train_ratio * total_images)

In [52]:
train_count

192

In [56]:
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('val'):
    os.makedirs('val')

In [58]:
for i, image_filename in enumerate(images):
    source_path = os.path.join('captured_images', image_filename)
    if i < train_count:
        destination_path = os.path.join('train', image_filename)
    else:
        destination_path = os.path.join('val', image_filename)
    shutil.copy(source_path, destination_path)
    print(f"Copied {image_filename} to {'train' if i < train_count else 'val'}")

Copied frame_241.jpg to train
Copied frame_100.jpg to train
Copied frame_86.jpg to train
Copied frame_82.jpg to train
Copied frame_208.jpg to train
Copied frame_97.jpg to train
Copied frame_179.jpg to train
Copied frame_34.jpg to train
Copied frame_175.jpg to train
Copied frame_247.jpg to train
Copied frame_71.jpg to train
Copied frame_52.jpg to train
Copied frame_127.jpg to train
Copied frame_200.jpg to train
Copied frame_244.jpg to train
Copied frame_41.jpg to train
Copied frame_152.jpg to train
Copied frame_56.jpg to train
Copied frame_12.jpg to train
Copied frame_228.jpg to train
Copied frame_14.jpg to train
Copied frame_125.jpg to train
Copied frame_240.jpg to train
Copied frame_40.jpg to train
Copied frame_114.jpg to train
Copied frame_120.jpg to train
Copied frame_128.jpg to train
Copied frame_158.jpg to train
Copied frame_50.jpg to train
Copied frame_109.jpg to train
Copied frame_117.jpg to train
Copied frame_110.jpg to train
Copied frame_230.jpg to train
Copied frame_93.jpg to

In [61]:
train = os.listdir('train')
val = os.listdir('val')

In [63]:
train_text = [filename.replace('.jpg', '.txt') for filename in train]
val_text = [filename.replace('.jpg', '.txt') for filename in val]

In [65]:
if not os.path.exists('train_labels'):
    os.makedirs('train_labels')
if not os.path.exists('val_labels'):
    os.makedirs('val_labels')

In [66]:
labels = os.listdir('captured_labels')

In [67]:
for text in train_text:
    if text in labels:
        departure = os.path.join('captured_labels', text)
        arrival = os.path.join('train_labels', text)
        shutil.copy(departure, arrival)

for text in val_text:
    if text in labels:
        departure = os.path.join('captured_labels', text)
        arrival = os.path.join('val_labels', text)
        shutil.copy(departure, arrival)

In [68]:
train_text_num = len(os.listdir('train_labels'))
val_text_num = len(os.listdir('val_labels'))
print(train_text_num)
print(val_text_num)

192
49


In [69]:
print(len(train))
print(len(val))

192
49
